In [1]:
from getpass import getpass

profile_name = 'default'
region_name = 'us-east-1'
aws_access_key_id = getpass('access id')
aws_secret_access_key = getpass('access key')

access id ········
access key ········


In [2]:
# Import libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates
import logging
import boto3
import pandas as pd
import pyarrow.parquet as pq
import io
import os
from botocore.exceptions import ClientError

In [3]:
#connect to s3
print('get_s3_client: profile_name=%s, region_name=%s' % (profile_name, region_name))

session = boto3.Session(profile_name=profile_name)
s3 = session.client('s3',
    region_name=region_name)

get_s3_client: profile_name=default, region_name=us-east-1


In [4]:
s3 = boto3.resource(
    service_name='s3',
    region_name= region_name,
    aws_access_key_id= aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key
)
buffer = io.BytesIO()

In [5]:
#view buckets
for bucket in s3.buckets.all():
    print(bucket.name)

training-prediction-s3


In [43]:
source = s3.Bucket('training-prediction-s3').Object('raw/run_ww_2020_d.parquet')
source.download_fileobj(buffer)
df_2020 = pd.read_parquet(buffer)

source2 = s3.Bucket('training-prediction-s3').Object('raw/run_ww_2019_d.parquet')
source2.download_fileobj(buffer)
df_2019 = pd.read_parquet(buffer)

df_combined = pd.concat([df_2019,df_2020])

OSError: Unexpected end of stream: Page was smaller (5002330) than expected (5016033)

In [44]:
df_combined.head()

,datetime,athlete,distance,duration,gender,age_group,country,major,datetime2,year,month,day,week_no_in_month,day_run,distance_rounded,athlete_month
0,2019-01-01,0,0.00,0.00,F,18 - 34,United States,CHICAGO 2019,2019-01-01,2019,1,1,1,0,0.0,0_1_2019
1,2019-01-01,1,5.27,30.20,M,35 - 54,Germany,BERLIN 2016,2019-01-01,2019,1,1,1,1,5.0,1_1_2019
2,2019-01-01,2,0.00,0.00,M,35 - 54,United Kingdom,"LONDON 2018,LONDON 2019",2019-01-01,2019,1,1,1,0,0.0,2_1_2019
3,2019-01-01,3,10.50,43.95,M,18 - 34,United Kingdom,LONDON 2017,2019-01-01,2019,1,1,1,1,10.0,3_1_2019
4,2019-01-01,4,9.66,48.65,M,35 - 54,United States,BOSTON 2017,2019-01-01,2019,1,1,1,1,10.0,4_1_2019


In [45]:
df_combined['datetime2'] = df_combined['datetime'].astype('string')
df_combined['year'] = df_combined['datetime2'].str.slice(0, 4)
df_combined['month'] = df_combined['datetime2'].str.slice(5, 7)
df_combined['day'] = df_combined['datetime2'].str.slice(8, )
df_combined['year'] = df_combined['year'].astype(int)
df_combined['month'] = df_combined['month'].astype(int)
df_combined['day'] = df_combined['day'].astype(int)
df_combined.dtypes

datetime            datetime64[ns]
athlete                      int64
distance                   float64
duration                   float64
gender                      object
age_group                   object
country                     object
major                       object
datetime2           string[python]
year                         int64
month                        int64
day                          int64
week_no_in_month             int64
day_run                      int64
distance_rounded           float64
athlete_month               object
dtype: object

In [46]:
import calendar
import numpy as np
calendar.setfirstweekday(6)

def get_week_of_month(year, month, day):
    x = np.array(calendar.monthcalendar(year, month))
    week_of_month = np.where(x==day)[0][0] + 1
    return(week_of_month)

In [47]:
df_combined['week_no_in_month'] = df_combined.apply(lambda x: get_week_of_month( x['year'], x['month'], x['day']), axis=1)

In [48]:
df_combined['day_run'] = df_combined.apply(lambda row: 1 if row['distance'] > 0 else 0, axis=1)

In [60]:
df_combined_agg_week = pd.DataFrame(df_combined.groupby(['athlete', 'month', 'year', 'week_no_in_month']).agg({"distance": sum, "duration": sum, "day_run": sum}))
df_combined_agg_week.head()

distance  duration  day_run
athlete month year week_no_in_month                             
0       1     2019 1                      0.0       0.0        0
                   2                      0.0       0.0        0
                   3                      0.0       0.0        0
                   4                      0.0       0.0        0
                   5                      0.0       0.0        0

In [61]:
df_combined_agg_week = df_combined_agg_week.reset_index()
df_combined_agg_week['pace'] = (df_combined_agg_week['duration'])/(df_combined_agg_week['distance'])

In [62]:
df_combined_agg_week['pace'] = df_combined_agg_week.pace.fillna(0)

In [63]:
df_combined_agg_week2 = pd.DataFrame(df_combined_agg_week.groupby(['athlete', 'month', 'year']).agg({"distance": 'mean', "pace": 'mean', "day_run": 'mean'}))
df_combined_agg_week2 = df_combined_agg_week2.reset_index()

In [64]:
df_athlete_ids = df_combined.groupby('athlete').agg({"age_group": "first","gender": "first", "country": "first"})
df_athlete_ids = df_athlete_ids.reset_index()

In [65]:
df_combined_athletes_months = df_combined_agg_week2.merge(df_athlete_ids, on='athlete', how='outer')
df_combined_athletes_months.head()

,athlete,month,year,distance,pace,day_run,age_group,gender,country
0,0,1,2019,0.0,0.000000,0.0,18 - 34,F,United States
1,0,1,2020,0.0,0.000000,0.0,18 - 34,F,United States
2,0,2,2019,0.0,0.000000,0.0,18 - 34,F,United States
3,0,2,2020,1.3,1.049744,0.2,18 - 34,F,United States
4,0,3,2019,0.0,0.000000,0.0,18 - 34,F,United States


## this is first half now for second half

In [66]:
df_combined['distance_rounded'] = df_combined.distance.round()

In [67]:
df_combined['athlete_month'] = df_combined["athlete"].astype(str) + "_" + df_combined["month"].astype(str) + "_" + df_combined["year"].astype(str)

In [102]:
value_counts_in_month = df_combined.groupby(['athlete_month', 'distance_rounded','year']).size().reset_index(name='count')
df_combined_athletes_months['athlete_month2'] = df_combined_athletes_months["athlete"].astype(str) + "_" + df_combined_athletes_months["month"].astype(str) + "_" + df_combined_athletes_months["year"].astype(str)
value_counts_in_month[['athlete', 'month', 'year']] = value_counts_in_month.athlete_month.str.split("_", expand = True)

In [103]:
value_counts_in_month['athlete'] = value_counts_in_month['athlete'].astype(int)
value_counts_in_month['month'] = value_counts_in_month['month'].astype(int)
value_counts_in_month['year'] = value_counts_in_month['year'].astype(int)
value_counts_in_month.dtypes

athlete_month        object
distance_rounded    float64
year                  int64
count                 int64
athlete               int64
month                 int64
dtype: object

In [104]:
def days_in_month(year, month):
    _, num_days = calendar.monthrange(year, month)
    return num_days

In [105]:
value_counts_in_month['days_in_month'] = value_counts_in_month.apply(lambda x: days_in_month(x['year'], x['month']), axis=1)
value_counts_in_month['weekly_frequency'] = value_counts_in_month['count'] / (value_counts_in_month['days_in_month'] / 7)

In [106]:
def month_name_from_number(month_number):
    if 1 <= month_number <= 12:
        return calendar.month_name[month_number]
    else:
        return None  # You can handle invalid month numbers as per your requirements



In [107]:
# Create a new 'month_name' column in the DataFrame
value_counts_in_month['current_month'] = value_counts_in_month['month'].apply(month_name_from_number)

value_counts_in_month.loc[value_counts_in_month['month'] > 1, 'previous_month'] = value_counts_in_month['month'] - 1
value_counts_in_month.loc[value_counts_in_month['month'] > 1, 'previous_year'] = value_counts_in_month['year']
value_counts_in_month.loc[value_counts_in_month['month'] == 1, 'previous_month'] = 12
value_counts_in_month.loc[value_counts_in_month['month'] == 1, 'previous_year'] = value_counts_in_month['year'] - 1


value_counts_in_month['previous_month'] = value_counts_in_month['previous_month'].astype('string')
value_counts_in_month['previous_year'] = value_counts_in_month['previous_year'].astype('string')
value_counts_in_month['athlete_month2'] = value_counts_in_month['athlete'].astype('string') + '_'+ value_counts_in_month['previous_month'].str[:-2]  + '_' +  value_counts_in_month['previous_year'].str[:-2]
value_counts_in_month2 = value_counts_in_month[['athlete_month', 'athlete_month2', 'athlete','current_month', 'distance_rounded', 'weekly_frequency']]

df_combined_athletes_prev_months = df_combined_athletes_months[['athlete_month2', 'distance', 'pace', 'day_run']]

In [108]:
value_counts_in_month.sample(10)

,athlete_month,distance_rounded,year,count,athlete,month,days_in_month,weekly_frequency,current_month,previous_month,previous_year,athlete_month2
1000001,15580_3_2019,5.0,2019,2,15580,3,31,0.451613,March,2.0,2019.0,15580_2_2019
1599320,18949_9_2019,34.0,2019,1,18949,9,30,0.233333,September,8.0,2019.0,18949_8_2019
1528496,18557_2_2020,11.0,2020,1,18557,2,29,0.241379,February,1.0,2020.0,18557_1_2020
3432695,2955_10_2020,11.0,2020,1,2955,10,31,0.225806,October,9.0,2020.0,2955_9_2020
1019964,1568_5_2019,5.0,2019,2,1568,5,31,0.451613,May,4.0,2019.0,1568_4_2019
5809447,9098_11_2019,5.0,2019,3,9098,11,30,0.700000,November,10.0,2019.0,9098_10_2019
584030,13247_2_2020,7.0,2020,1,13247,2,29,0.241379,February,1.0,2020.0,13247_1_2020
1987735,2120_10_2020,8.0,2020,1,2120,10,31,0.225806,October,9.0,2020.0,2120_9_2020
3937701,32563_8_2019,10.0,2019,1,32563,8,31,0.225806,August,7.0,2019.0,32563_7_2019
4708253,36847_1_2020,6.0,2020,6,36847,1,31,1.354839,January,12.0,2019.0,36847_12_2019


In [109]:
df_combined_athletes_identities = df_combined_athletes_months[['athlete', 'age_group','gender','country']]
value_counts_in_month2['athlete_month2'] = value_counts_in_month2['athlete_month2'].astype('object')

df_combined_athletes_identities['athlete'] = df_combined_athletes_identities['athlete'].astype('int')
df_combined_athletes_identities.dtypes

/tmp/ipykernel_10821/2242505681.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_counts_in_month2['athlete_month2'] = value_counts_in_month2['athlete_month2'].astype('object')
/tmp/ipykernel_10821/2242505681.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined_athletes_identities['athlete'] = df_combined_athletes_identities['athlete'].astype('int')


athlete       int64
age_group    object
gender       object
country      object
dtype: object

In [110]:
merged_a = value_counts_in_month2.merge(df_combined_athletes_prev_months, on='athlete_month2', how='left')
merged_b = merged_a.merge(df_combined_athletes_identities.drop_duplicates(subset=['athlete']),on='athlete', how='left')

In [111]:
df_combined_athletes_prev_months.head()

,athlete_month2,distance,pace,day_run
0,0_1_2019,0.0,0.000000,0.0
1,0_1_2020,0.0,0.000000,0.0
2,0_2_2019,0.0,0.000000,0.0
3,0_2_2020,1.3,1.049744,0.2
4,0_3_2019,0.0,0.000000,0.0


In [112]:
value_counts_in_month2.head()

,athlete_month,athlete_month2,athlete,current_month,distance_rounded,weekly_frequency
0,0_10_2019,0_9_2019,0,October,0.0,5.645161
1,0_10_2019,0_9_2019,0,October,4.0,0.225806
2,0_10_2019,0_9_2019,0,October,6.0,0.225806
3,0_10_2019,0_9_2019,0,October,8.0,0.225806
4,0_10_2019,0_9_2019,0,October,9.0,0.225806


In [113]:
new_column_names = {
    'distance': 'prev_month_weekly_km',
    'pace': 'prev_month_weekly_pace',
    'day_run': 'prev_month_weekly_days_run',
    'age_group': 'age_bucket',
    'distance_rounded': 'run_distance_rounded',
}

merged_b = merged_b.rename(columns=new_column_names)
merged_b.head()

,athlete_month,athlete_month2,athlete,current_month,run_distance_rounded,weekly_frequency,prev_month_weekly_km,prev_month_weekly_pace,prev_month_weekly_days_run,age_bucket,gender,country
0,0_10_2019,0_9_2019,0,October,0.0,5.645161,47.48,6.185043,3.4,18 - 34,F,United States
1,0_10_2019,0_9_2019,0,October,4.0,0.225806,47.48,6.185043,3.4,18 - 34,F,United States
2,0_10_2019,0_9_2019,0,October,6.0,0.225806,47.48,6.185043,3.4,18 - 34,F,United States
3,0_10_2019,0_9_2019,0,October,8.0,0.225806,47.48,6.185043,3.4,18 - 34,F,United States
4,0_10_2019,0_9_2019,0,October,9.0,0.225806,47.48,6.185043,3.4,18 - 34,F,United States


In [114]:
input_data_combined = merged_b[['athlete', 'prev_month_weekly_km', 'prev_month_weekly_pace', 'prev_month_weekly_days_run','age_bucket', 'gender', 'country', 'current_month', 'run_distance_rounded', 'weekly_frequency']]
input_data_combined_nozeros = input_data_combined[input_data_combined['run_distance_rounded'] > 0]

#remove any missings from the data
input_data_combined_nozeros = input_data_combined_nozeros[input_data_combined_nozeros["prev_month_weekly_km"].notna()]

In [115]:
input_data_combined_nozeros.head()

,athlete,prev_month_weekly_km,prev_month_weekly_pace,prev_month_weekly_days_run,age_bucket,gender,country,current_month,run_distance_rounded,weekly_frequency
1,0,47.48,6.185043,3.4,18 - 34,F,United States,October,4.0,0.225806
2,0,47.48,6.185043,3.4,18 - 34,F,United States,October,6.0,0.225806
3,0,47.48,6.185043,3.4,18 - 34,F,United States,October,8.0,0.225806
4,0,47.48,6.185043,3.4,18 - 34,F,United States,October,9.0,0.225806
5,0,47.48,6.185043,3.4,18 - 34,F,United States,October,10.0,0.225806


In [116]:
try:
    os.mkdir('../../raw')
except:
    print('dir already exists')
input_data_combined_nozeros.to_parquet('../../raw/input_runners_all.parquet', index=False)

dir already exists
